# 라이브러리

In [19]:
! pip3 install koreanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# 정의

📊 퍼널 단계 정의 (채용 플랫폼 기준)
1. 랜딩 단계 (Landing / 탐색 진입)
    - 관련 URL: home, search, suggest, jobs, companies, people
    - 목적: 플랫폼에 들어와서 정보를 둘러보기 시작함.

2. 정보 탐색 단계 (View Details)
    - 관련 URL: jobs/job_id, companies/company_id, people/id, jobs/id/other_jobs
    - 목적: 개별 채용공고나 기업/인물의 상세 정보 열람

3. 행동 유도 단계 (Engagement / Action Intent)
    - 관련 URL: continue, signup, email_verify, verify_phone, setting
    - 목적: 회원가입을 시작하거나, 지원을 위한 절차 진행

4. 지원 단계 (Apply)
    - 관련 URL: apply, applications/job_id, @user_id?action=request_approval_complete
    - 목적: 실제 채용공고에 지원하는 행동

5. 완료/성과 단계 (Success / Conversion)
    - 관련 URL: password_reset, help, zip_code, admin, app, pricing
    - 목적: 전환 이후 지속 이용 또는 추가 행동을 보임

🔁 전환/이탈률 분석 설계
각 단계의 유입 사용자 수를 기반으로:

|퍼널 단계|방문자 수|전환률 (%)|이탈률 (%)|
|---|---|---|---|
|랜딩|A|-|((A - B) / A) * 100|
|정보 탐색|B|(B / A) * 100|((B - C) / B) * 100|
|행동 유도|C|(C / B) * 100|((C - D) / C) * 100|
|지원|D|(D / C) * 100|((D - E) / D) * 100|
|전환 완료|E|(E / D) * 100|-|

# 데이터 불러오기

In [21]:
log_2022_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/log_2022.csv"
log_2022_df = pd.read_csv(log_2022_path)
log_2022_df.head(2)

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454 UTC,2022-06-23,200,GET
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741 UTC,2022-06-23,200,POST


In [22]:
# 1. UTC 제거
log_2022_df["timestamp_clean"] = log_2022_df["timestamp"].str.replace(" UTC", "", regex=False)

# 2. datetime으로 변환
log_2022_df["timestamp_clean"] = pd.to_datetime(
    log_2022_df["timestamp_clean"],
    format='%Y-%m-%d %H:%M:%S.%f',
    errors='coerce'
)

# 3. NaT가 생긴 경우, 포맷이 다른 문자열을 다시 처리 (마이크로초 없는 경우 등)
mask_failed = log_2022_df["timestamp_clean"].isna()
log_2022_df.loc[mask_failed, "timestamp_clean"] = pd.to_datetime(
    log_2022_df.loc[mask_failed, "timestamp"].str.replace(" UTC", "", regex=False),
    format="%Y-%m-%d %H:%M:%S",
    errors="coerce"
)

# 4. 정리
log_2022_df["timestamp"] = log_2022_df["timestamp_clean"]
log_2022_df.drop(columns="timestamp_clean", inplace=True)

In [ ]:
# UTM 파라미터 확인
log_2022_df[log_2022_df["URL"].str.contains("utm_", case=False, na=False)]

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
20,20,8a7e70cd-1630-4a8a-8303-0706af3a7908,@user_id?utm_source=notification&utm_medium=em...,2022-06-23 16:35:25.501568,2022-06-24,200,GET
129,129,8a7e70cd-1630-4a8a-8303-0706af3a7908,r?type=experience_verify&c=87c2170d-c7e5-4bb8-...,2022-06-22 16:26:09.919652,2022-06-23,302,GET
168,168,8a7e70cd-1630-4a8a-8303-0706af3a7908,jobs/id/id_title?utm_campaign=google_jobs_appl...,2022-06-22 16:21:07.392774,2022-06-23,200,GET
255,255,207a8876-2a6c-4137-9347-476ce719c541,@user_id?utm_source=notification&utm_medium=em...,2022-06-22 22:10:33.194900,2022-06-23,200,GET
287,287,207a8876-2a6c-4137-9347-476ce719c541,@user_id?utm_source=notification&utm_medium=em...,2022-06-02 22:14:13.023414,2022-06-03,200,GET
...,...,...,...,...,...,...,...
10054105,10054105,d491cf04-1408-4f1a-bc14-3dccccdc05a4,setting?utm_source=notification,2022-08-19 14:45:52.762488,2022-08-19,200,GET
10054107,10054107,b26bf8fc-58d6-4a7c-8dfa-b05c37bc3570,setting?utm_source=notification&utm_medium=ema...,2022-07-11 18:19:19.566660,2022-07-12,200,GET
10054113,10054113,a4295723-f94c-4f3a-8a48-37c1b8e4850a,jobs/id/id_title?utm_campaign=google_jobs_appl...,2022-02-10 06:55:11.196278,2022-02-10,200,GET
10054118,10054118,d2b90cda-7e0d-454f-8a64-070048551e14,setting?utm_source=notification&utm_medium=ema...,2022-07-27 01:47:23.149828,2022-07-27,200,GET


In [ ]:
# UTM 파라미터 제거
from urllib.parse import urlparse

def remove_query_params(url):
    try:
        return urlparse(url).path.lower()
    except:
        return str(url).lower()

log_2022_df["clean_url"] = log_2022_df["URL"].apply(remove_query_params)

# 퍼널 단계 부여

In [ ]:
# clean_url 만들기
log_2022_df["clean_url"] = log_2022_df["clean_url"].str.lower()

# 퍼널 단계 정의 (4번째 수정함...)
# funnel_stage를 나누는 과정에서 조건이 누락됐거나 너무 좁게 정의됐을 경우 (jobs/ 대신 api/jobs/job_title처럼 되어 있음)
funnel_stages = {
    "landing": ["home", "search", "suggest", "jobs", "companies", "people"],
    "view_details": ["job", "company", "people", "other_jobs"],
    "engagement": ["continue", "signup", "email_verify", "verify_phone", "setting"],
    "apply": ["apply", "applications", "request_approval_complete", "@user_id"],
    "converted": ["password_reset", "help", "zip_code", "admin", "app", "pricing"]
}

# 퍼널 단계 부여 함수
def assign_funnel_stage(url):
    url = str(url).lower()
    for stage, keywords in funnel_stages.items():
        if any(k in url for k in keywords):
            return stage
    return "other"

# funnel_stage 컬럼 생성
log_2022_df["funnel_stage"] = log_2022_df["clean_url"].apply(assign_funnel_stage)

log_2022_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,other


## other 단계 제외

In [ ]:
# other 행만 모아 새로운 데이터프레임 생성
out_of_stage_df = log_2022_df[log_2022_df["funnel_stage"] == "other"]
out_of_stage_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,other
5,5,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/education/template?type=apply&_=1...,2022-06-22 16:25:42.416476,2022-06-23,200,GET,api/users/id/education/template,other
8,8,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/experience/form?type=apply,2022-06-22 16:34:11.692417,2022-06-23,200,POST,api/users/id/experience/form,other


In [ ]:
# other 행을 제외한 나머지
log_df = log_2022_df[log_2022_df["funnel_stage"] != "other"]
log_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
6,6,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/people/job_title?name=data en&_=165...,2022-06-22 16:20:10.851840,2022-06-23,200,GET,api/search/people/job_title,landing
7,7,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/specialty?name=bento,2022-06-22 16:33:43.647159,2022-06-23,200,GET,api/search/specialty,landing
10,10,8a7e70cd-1630-4a8a-8303-0706af3a7908,setting,2022-06-26 08:11:33.254330,2022-06-26,200,GET,setting,engagement


# 각 단계별 사용자 수

UTM 파라미터 제거 전

|단계|사용자 수|
|---|---|
|landing|15934|
|view_details|6812|
|engagement|9872|
|apply|15776|
|success|7663|

-> 제거 후 결과와 큰 차이 없음

In [ ]:
# 각 퍼널 단계별 고유 사용자 수 계산
funnel_user_counts = log_df.groupby("funnel_stage")["user_uuid"].nunique()

# 순서를 맞춰 정렬
funnel_order = ["landing", "view_details", "engagement", "apply", "converted"]
funnel_user_counts = funnel_user_counts.reindex(funnel_order).fillna(0).astype(int)

# A, B, C, D, E에 할당
A, B, C, D, E = funnel_user_counts.values

# 단계별 사용자 수를 데이터프레임으로 정리
stage_user_counts_df = pd.DataFrame({
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "사용자 수": [A, B, C, D, E]
})

stage_user_counts_df

,단계,사용자 수
0,landing,15842
1,view_details,5993
2,engagement,10691
3,apply,15747
4,converted,7662


# 전환률 계산

In [ ]:
funnel_metrics = {
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "방문자 수": [A, B, C, D, E],
    "전환률 (%)": ["-", round((B / A) * 100, 2) if A else 0, round((C / B) * 100, 2) if B else 0, round((D / C) * 100, 2) if C else 0, round((E / D) * 100, 2) if D else 0],
    "이탈률 (%)": [round(((A - B) / A) * 100, 2) if A else "-", round(((B - C) / B) * 100, 2) if B else "-", round(((C - D) / C) * 100, 2) if C else "-", round(((D - E) / D) * 100, 2) if D else "-", "-"]
}

funnel_df = pd.DataFrame(funnel_metrics)
funnel_df

,단계,방문자 수,전환률 (%),이탈률 (%)
0,landing,15842,-,62.17
1,view_details,5993,37.83,-78.39
2,engagement,10691,178.39,-47.29
3,apply,15747,147.29,51.34
4,converted,7662,48.66,-


# 인사이트

## Landing -> View Details

- 전환률: 42.75%
- 이탈률: 57.25%
    - 약 절반 이상이 상세 내용을 보지 않고 떠남 -> 랜딩 페이지 품질 또는 추천 콘텐츠/검색 결과 적합성 문제

## 2. View Details -> Engagement

- 전환률: 144.92% (비정상적으로 높음)
    - 이는 단계 중복 또는 URL 라벨링 오류 가능성 있음
        - 사용자가 View 없이 바로 Signup한 경우
        - Engagement URL이 너무 포괄적하게 잡혀 있을 가능성 있음

## 3. Engagement -> Apply

- 전환률: 159.81%
    - 역시 중간 단계가 일부 사용자에게 스킵되었거나, Apply로 바로 가는 구조

## 4. Apply -> Success

- 전환률: 48.57%
    - 실제 전환까지 이어진 비율로 괜찮은 수치
    - 절반 정도는 지원 후 추가 행동 없이 이탈
    - 알림, 후속 안내 UX 강화 필요

## 추천 액션

- 퍼널 단계 정의 정교화
    - view_details와 engagement 사이 흐름 점검
    - 혹시 중복이 있거나 불필요한 포함 키워드가 있는지 확인

- 비정상 전환률(100% 이상)
    - 중복 방문자, 여러 단계 누락된 사용자 흐름을 따로 분석 필요

- 실 사용자 흐름 시각화
    - Sankey Diagram이나 Funnel Chart로 각 사용자 흐름 시각화

- 세그먼트별 퍼널 분석
    - 예: job_seeker, employer 각각의 퍼널 전환율 비교 -> 맞춤 전략 가능

# 시각화

## Sankey Diagram (Plotly)

In [ ]:
import plotly.graph_objects as go

# 사용자별 퍼널 단계 시퀀스 정리
user_funnel = log_df.sort_values(by=["user_uuid", "timestamp"])
user_funnel = user_funnel.groupby("user_uuid")["funnel_stage"].apply(lambda x: list(dict.fromkeys(x)))  # 중복 제거

# 전이 단계 추출
from collections import Counter
transitions = Counter()

for stages in user_funnel:
    for i in range(len(stages)-1):
        transitions[(stages[i], stages[i+1])] += 1

# 고유 단계 리스트
labels = list({s for pair in transitions.keys() for s in pair})
label_index = {k: i for i, k in enumerate(labels)}

# 출발, 도착, 값 정의
source = [label_index[src] for src, tgt in transitions.keys()]
target = [label_index[tgt] for src, tgt in transitions.keys()]
value = list(transitions.values())

# Sankey 그리기
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    ))])

fig.update_layout(title_text="사용자 퍼널 단계 이동 Sankey Diagram", font_size=12)
fig.show()

## Funnel Chart (단계별 전환 시각화)

In [ ]:
import plotly.express as px

stage_user_counts_df = pd.DataFrame({
    "funnel_stage": ["landing", "view_details", "engagement", "apply", "converted"],
    "users": [A, B, C, D, E]
})

fig = px.funnel(stage_user_counts_df, x='users', y='funnel_stage', title="사용자 퍼널 단계별 전환")
fig.show()

# 추가

## UTM 파라미터 제거 전 후 결과가 동일한 이유

1. UTM이 포함된 URL은 대부분 중복 방문일 가능성 있음
- utm_ 파라미터가 붙는 URL은 보통 마케팅 채널을 통한 유입 추적이 목적이기 때문
    - 기존 사용자가 여러 번 들어올 때 다른 utm_이 붙어서 같은 페이지를 반복 방문하는 경우가 많음
    - 퍼널 분석에서는 user_uuid 기준으로 고유 사용자 수를 세기 때문에, 똑같은 사람이 여러 번 utm 붙은 URL로 들어와도 1명으로만 집계됨
    - 즉, 전체 퍼널 흐름의 경로 자체를 바꾸지 않았기 때문에 전환률은 그대로일 수 있음

2. 쿼리스트링을 제거해도 퍼널 분류 기준 키워드가 동일하게 인식됨
- 퍼널 분류 함수는 쿼리스트링을 보지 않고 경로(path)만으로 분류했기 때문에, UTM 제거 여부와 관계없이 같은 퍼널 단계로 인식

3. 쿼리 제거 전과 후의 clean_url 값이 이미 같았던 경우가 많음
- URL을 쿼리까지 포함한 전체 문자열로 사용한 게 아니라, 퍼널 분류에 쓰기 전에 이미 .str.lower() 같은 걸 써서 경로만 추출한 값들과 비교했다면, 이미 분석이 경로 중심으로 되어 있었기 때문에 UTM 파라미터 제거는 이미 반영되어 있었던 거나 마찬가지였을 수 있음

## 전환률 계산 적절한가?

- 퍼널 전환률은 이전 단계 대비 다음 단계로 몇 %의 사용자가 넘어갔는지 확인
    - 예시
        - A (landing): 10,000명
        - B (view_details): 6,000명
        - ```전환률 = B/A X 100 = 6,000/10,000 X 100 = 60%```
            - landing한 사람 중 60%가 view_details 단계로 넘어감

- 주의할 점
    - 해당 계산법은 퍼널 구조가 선형적일 때 가장 잘 들어맞음
    - 사용자가 중간 단계를 건너뛸 수 있는 비선형적 구조일 때는
    - 전환률이 100%를 넘어가거나, 해석의 차이가 발생할 수 있음
        - 퍼널을 유연하게 해석할 필요 있음
            - 예시) 전환률이 100%를 넘으면 이전 단계를 건너뛰고 해당 단계로 유입된 사용자가 있다는 뜻
        - 따라서 해석할 때 사용자 흐름을 시각화(Sankey 등)해서 보조적으로 파악하는 게 중요

## 이탈률이 음수로 나오는 경우

- 이탈률이 음수로 나온다는 건 원래 개념적으로는 잘못된 상황
- 데이터나 계산 방식에 따라 나올 수 있는 특이 케이스 중 하나

```이탈률 = ((이전 단계 방문자 수 - 현재 단계 방문자 수) / 이전 단계 방문자 수) * 100```

- 현재 단계의 사용자 수가 이전 단계보다 많아지는 경우 이탈률이 음수가 됨

- 유저가 퍼널을 건너뛰고 들어온 경우
    - 어떤 유저가 검색(landing)을 하지 않고 바로 apply 페이지로 접근
    - 이럴 경우 이전 단계에 기록되지 않고도 다음 단계에서 활동

- 퍼널 단계 분류 로직의 오류
    - URL이 잘못 분류되어, 특정 단계에 사용자 수가 실제보다 적게 잡히는 경우

- 이벤트 중복/로깅 시점의 문제
    - 로그가 제대로 시간 순서대로 정렬되지 않았거나
    - 이벤트가 잘못 중복 기록되어 다음 단계 유입이 과도하게 잡힘

- 사용자 세션 구분 오류
    - 여러 세션 또는 여러 사용자의 활동이 섞여버려 잘못 카운트됨

### 보정 아이디어

- 퍼널 단계를 순서 기준으로 필터링해서, 이전 단계보다 이후 단계에 있는 사용자만 포함
- 로그 시간 순서를 보장 (sort_values(by=["user_uuid", "timestamp"]))
- 중복 제거 또는 고유 세션 기반 분석
- 퍼널 정의/분류를 재점검 (URL 패턴 누락 체크)

# 특정 유저 하나만 확인

In [ ]:
log_2022_df["user_uuid"].unique()

array(['8a7e70cd-1630-4a8a-8303-0706af3a7908',
       '207a8876-2a6c-4137-9347-476ce719c541',
       'd7c84302-545f-4714-bcd1-0cc4328a0f89', ...,
       'f9387b22-2d7d-4267-8671-2279e2d784b9',
       '849ff1bb-6280-46f4-baaa-71d1d0796b5b',
       '940b883f-a385-416c-94e2-b5604a78506c'], dtype=object)

In [ ]:
user_1 = log_2022_df[log_2022_df["user_uuid"] == "207a8876-2a6c-4137-9347-476ce719c541"]

user_1["URL"].unique()

array(['api/users/id/template', 'search?keywords=생물정보학', 'suggest?q=생물',
       'suggest?q=생', 'search?keywords=마이크로바이옴', 'suggest?q=',
       'companies/company_id', 'api/search/template?keywords=마이크로바이옴&q=',
       'api/companies/id/view', 'api/search/template?keywords=@지니너스&q=',
       'api/jobs/job_title?keywords=bioinformatics&page=&q=',
       '@user_id?utm_source=notification&utm_medium=email&utm_campaign=recommend_people&utm_content=user_342499_profile',
       'companies/company_id/jobs', 'suggest?q=생물정보학', 'suggest?q=ㅅ',
       'suggest?q=생물ㅈ', 'suggest?q=마이크로바이옴', 'suggest?q=김',
       'suggest?q=김ㅁ', 'api/people/template?keywords=김민재&page=&q=',
       'api/search/template?keywords=bioninformatics&q=',
       'suggest?q=bioi', 'suggest?q=bioninforma', nan, 'suggest?q=bioin',
       'suggest?q=ㅇ', '@user_id',
       'api/search/template?keywords=bioinformatics&q=',
       'api/jobs/id/other_jobs?offset=0&limit=5', 'people?company=24720',
       'suggest?q=b', 'signup/step1/na

In [ ]:
pd.set_option('display.max_rows', None)        # 행 제한 없음
pd.set_option('display.max_columns', None)     # 열 제한 없음
pd.set_option('display.width', None)           # 가로 너비 제한 없음
pd.set_option('display.max_colwidth', None)    # 열 내용 길이 제한 없음

user_1_sorted = user_1.sort_values(by="timestamp").reset_index(drop=True)

user_1_sorted[["timestamp", "URL"]]

,timestamp,URL
0,2022-01-06 02:03:30.711295,@user_id?utm_source=notification&utm_medium=email&utm_campaign=recommend_people&utm_content=user_425596_profile
1,2022-01-06 02:03:33.559005,api/users/id/template
2,2022-01-06 02:04:04.943259,@user_id
3,2022-01-06 02:04:07.581707,api/users/id/template
4,2022-02-07 04:47:04.417295,NaN
5,2022-02-07 04:47:06.717872,NaN
6,2022-02-07 04:47:18.378761,suggest?q=
7,2022-02-07 04:47:20.243449,suggest?q=ㅠ
8,2022-02-07 04:47:21.915613,suggest?q=bio
9,2022-02-07 04:47:22.837149,suggest?q=bioninfor


In [ ]:
# 설정을 기본값으로 리셋
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

# 전처리 데이터 추출

In [23]:
# UTM 파라미터 값을 가진 행만 추출
utm_2022_df = log_2022_df[log_2022_df['URL'].str.contains('utm_', na=False)]

# 'api/'로 시작하는 행만 추출
api_2022_df = log_2022_df[log_2022_df['URL'].str.startswith('api/', na=False)]

In [26]:
utm_2022_df.to_csv('utm_2022_df.csv', index=False, encoding='utf-8-sig')
api_2022_df.to_csv('api_2022_df.csv', index=False, encoding='utf-8-sig')

In [27]:
from urllib.parse import urlparse, parse_qs
from collections import Counter

#  URL 의미 기반 태깅 함수
def map_funnel_stage(url):
    url = str(url).lower()

    if pd.isna(url) or url.strip() == '':
        return '기타'

    if 'search' in url or 'keywords' in url or 'suggest' in url or 'q=' in url:
        return '검색'
    elif 'job_title' in url or 'jobs/id' in url:
        return '공고 보기'
    elif 'companies' in url or 'company_id' in url:
        return '기업 보기'
    elif 'template' in url or 'career' in url or 'experience' in url or 'profile' in url:
        return '이력서 작성'
    elif 'apply/step' in url:
        return '지원 시작'
    elif 'applications' in url:
        return '지원 완료'
    elif 'notifications/mark_read' in url:
        return '알림 확인'
    elif '@user_id' in url or 'follower' in url or 'following' in url:
        return '유저 프로필'
    elif 'setting' in url or 'timeline' in url:
        return '설정'
    elif 'signup' in url or 'password_reset' in url:
        return '회원가입'
    else:
        return '기타'

# URL 쿼리 파라미터 추출
def extract_query_keywords(url):
    try:
        parsed = urlparse(str(url))
        query = parse_qs(parsed.query)
        keywords = []
        for k, v in query.items():
            if k in ['q', 'keywords', 'name', 'type', 'job', 'offset']:
                keywords.append(f"{k}={','.join(v)}")
        return '&'.join(keywords)
    except:
        return ''

# 의미 기반 컬럼 생성
log_2022_df['funnel_stage'] = log_2022_df['URL'].apply(map_funnel_stage)
log_2022_df['query_tags'] = log_2022_df['URL'].apply(extract_query_keywords)

# 분석용 테이블 구성
behavior_table = log_2022_df[['user_uuid', 'timestamp', 'URL', 'funnel_stage', 'query_tags']].sort_values(by=['user_uuid', 'timestamp'])
behavior_table = behavior_table[
    ~behavior_table['URL'].str.contains('utm', case=False, na=False) &
    ~behavior_table['URL'].str.startswith('api/', na=False)
].reset_index(drop=True)

In [29]:
behavior_table.to_csv('log_2022_df.csv', index=False, encoding='utf-8-sig')